# Training a Baseline Model: GPT-2 fine-tuned for NLI using a random subset of size 190k of SNLI train data

### 1. Imports and Global Settings

In [1]:
from datasets import load_dataset, disable_caching
from tqdm.notebook import tqdm
from transformers import GPT2ForSequenceClassification, GPT2TokenizerFast, DataCollatorWithPadding, set_seed
import torch
from torch.nn.functional import one_hot
from utils_ import tokenize, train_classifier
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)
disable_caching()

### 2. Pre-Processing
- Get SNLI Dataset (Train fold) and shuffle it
- Remove instances without gold standard labels, i.e., label = -1
- Subset data to get 190k samples
- One-hot encoding for labels
- Tokenise data

In [2]:
train = load_dataset('snli', split = 'train').shuffle(42)
train = train.filter(lambda x: x['label'] != -1).map( \
    lambda x: {'label': one_hot(torch.tensor(x['label']), 3).type(torch.float32).numpy()}, \
    batched = True)
train = train.select(range(190000))

Reusing dataset snli (/home/shana92/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

In [3]:
# set up tokeniser
# padding to left because GPT2 uses last token for prediction
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", padding_side = 'left', \
                                              padding = True, truncation = True)
tokenizer.pad_token = tokenizer.eos_token # pad with 'eos' token

In [4]:
# tokenize data
train = train.map(lambda x: tokenize(tokenizer, x['premise'] + '|' + x['hypothesis']))
len_bef_exclusion = len(train)

# exclude instances with > 128 tokens
train = train.filter(lambda x: x['exclude'] == False)
len_aft_exclusion = len(train)

# print message if instances were in fact excluded
if len_bef_exclusion - len_aft_exclusion > 0:
    
    print(f'{len_bef_exclusion - len_aft_exclusion} ' + \
          f'({(en_bef_exclusion/len_aft_exclusion - 1):>2f}%) sequences excluded')

  0%|          | 0/190000 [00:00<?, ?ex/s]

  0%|          | 0/190 [00:00<?, ?ba/s]

In [5]:
# keep only needed columns, set data format to PyTorch
train.set_format(type = 'torch', columns = ['label', 'input_ids', 'attention_mask'])

### 3. Model

In [6]:
# set up data collator - https://huggingface.co/docs/transformers/main_classes/data_collator
# this is a (callable) helper object that sends batches of data to the model
data_collator = DataCollatorWithPadding(tokenizer, padding = 'max_length', \
                                         return_tensors = 'pt', max_length = 128)

In [7]:
# hyper-parameters for model training - set to match Le et al. (2020) - https://arxiv.org/abs/2002.04108
batch_size = 92
lr = 1e-5

In [8]:
# set up a dataloader (batch generator)
dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, \
                                 shuffle=True, collate_fn=data_collator) # batch size constrained by GPU memory

In [9]:
# instantiate GPT2 classifier model
model = GPT2ForSequenceClassification.from_pretrained("gpt2", 
                                  num_labels=3,
                                  problem_type="multi_label_classification")
model.config.pad_token_id = model.config.eos_token_id # specify pad_token used by tokenizer

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# set up optimizer (loss function in-built)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [11]:
# Train and Save model
torch.save(train_classifier(model, dataloader, optimizer, device), 'baseline_random_190k.pth')

  0%|          | 0/2066 [00:00<?, ?it/s]

loss: 1.573201  [    0/190000]
loss: 0.644930  [18952/190000]
loss: 0.646766  [37904/190000]
loss: 0.614586  [56856/190000]
loss: 0.613722  [75808/190000]
loss: 0.567028  [94760/190000]
loss: 0.525164  [113712/190000]
loss: 0.493526  [132664/190000]
loss: 0.501254  [151616/190000]
loss: 0.488926  [170568/190000]
loss: 0.442465  [189520/190000]
Epoch average loss: 0.5608803629875183


  0%|          | 0/2066 [00:00<?, ?it/s]

loss: 0.406714  [    0/190000]
loss: 0.446862  [18952/190000]
loss: 0.458301  [37904/190000]
loss: 0.385604  [56856/190000]
loss: 0.423286  [75808/190000]
loss: 0.344070  [94760/190000]
loss: 0.332827  [113712/190000]
loss: 0.360942  [132664/190000]
loss: 0.416128  [151616/190000]
loss: 0.439771  [170568/190000]
loss: 0.426085  [189520/190000]
Epoch average loss: 0.41300904750823975


  0%|          | 0/2066 [00:00<?, ?it/s]

loss: 0.332920  [    0/190000]
loss: 0.396391  [18952/190000]
loss: 0.357318  [37904/190000]
loss: 0.329222  [56856/190000]
loss: 0.376615  [75808/190000]
loss: 0.326639  [94760/190000]
loss: 0.337120  [113712/190000]
loss: 0.428314  [132664/190000]
loss: 0.307374  [151616/190000]
loss: 0.204890  [170568/190000]
loss: 0.336387  [189520/190000]
Epoch average loss: 0.3545190095901489
Done!
